<h1> Lecture 27

Data Science 8, Summer 2021 </h1>

In [ ]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [ ]:
# Some functions for plotting. You don't have to understand how any
# of the functions in this cell work, since they use things we 
# haven't learned about in Data 8.

def resize_window(lim=3.5):
    plots.xlim(-lim, lim)
    plots.ylim(-lim, lim)
    
def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='r'):
    y = x*slope + intercept
    plots.plot(x, y, color=color)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plots.plot(x, y, color=color)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    return x, y

def r_table(r):
    """
    Generate a table of 1000 x,y data points in standard units
    whose correlation is approximately equal to r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

## Slope & Intercept

In [ ]:
def standard_units(x):
    """Converts an array x to standard units"""
    return (x - np.mean(x)) / np.std(x)

def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)

In [ ]:
def slope(t, x, y):
    r = correlation(t, x, y)
    sd_x = np.std(t.column(x))
    sd_y = np.std(t.column(y))
    return r*sd_y/sd_x
    
def intercept(t, x, y):
    mean_x = np.mean(t.column(x))
    mean_y = np.mean(t.column(y))
    return mean_y - slope(t, x, y) * mean_x

In [ ]:
# r_table generates data for x and y such that the correlation of x and y is 0.5
example = r_table(0.5)

In [ ]:
example

What do you expect the slope and intercept to be?

In [ ]:
slope(example, 'x', 'y')

In [ ]:
intercept(example, 'x', 'y')

## Discussion Question

In [ ]:
mean_midterm = 70
sd_midterm = 10

mean_final = 50
sd_final =12
r = 0.75

slope_ex = r * sd_final/sd_midterm
int_ex = mean_final - slope_ex * mean_midterm

slope_ex * 90 + int_ex

## Movies data

Let's predict Domestic Gross from Budget

In [ ]:
movies = Table.read_table('movies.csv')
movies.show(11)

In [ ]:
cash = movies.select("Budget", "Domestic Gross")
cash

First, let's visualize the data

In [ ]:
cash.scatter('Budget','Domestic Gross')

Let's use nearest neighbors regression

In [ ]:
def predict_gross_nn(b):
    """Return a prediction of the domestic gross for a movie 
    with a budget of b
    
    The prediction is the average domestic gross of the movies
    whose budget is in the range b plus or minus $20 million dollars.
    """
    
    close_points = cash.where('Budget', are.between(b-20, b+20))
    return np.mean(close_points.column("Domestic Gross"))   

In [ ]:
cash_with_predictions = cash.with_column(
    'NN Prediction', cash.apply(predict_gross_nn, 'Budget')
    )

In [ ]:
cash_with_predictions.scatter('Budget')

Let's see our prediction for Domestic Gross for a movie with a Budget of $200 million

In [ ]:
nn_prediction = predict_gross_nn(200)
nn_prediction

Now let's try linear regression

In [ ]:
cash_slope = slope(cash, "Budget", "Domestic Gross")
cash_intercept = intercept(cash, "Budget", "Domestic Gross")
cash_slope, cash_intercept

Let's see our prediction for Domestic Gross for a movie with a Budget of $200 million using linear regression.

In [ ]:
linear_prediction = cash_slope * 200 + cash_intercept
linear_prediction

In [ ]:
linear_predictions = cash_slope * cash.column("Budget") + cash_intercept
cash_with_predictions.with_column("Linear Prediction", linear_predictions).scatter("Budget")

In [ ]:
cash_with_predictions.with_column("Linear Prediction", linear_predictions).scatter("Budget")

# plot a line (out of scope)
# draw line is a function defined at the top of the notebook
draw_line(cash_slope, cash_intercept, make_array(-100, 500), color='g')
plots.xlim([0, 400]);

## Least Squares

### Error in Estimation

In [ ]:
def demographics_errors(slope, intercept):
    # Use four convenient points from the original data
    sample = [[14.7, 33995], [19.1, 61454], [50.7, 71183], [59.5, 105918]]
    demographics.scatter('College%', 'Median Income', alpha=0.5)
    xlims = make_array(5, 75)
    # Plot a line with the slope and intercept you specified:
    plots.plot(xlims, slope * xlims + intercept, lw=4)
    # Plot red lines from each of the four points to the line
    for x, y in sample:
        plots.plot([x, x], [y, slope * x + intercept], color='r', lw=4)

In [ ]:
demographics = Table.read_table('district_demographics2016.csv')
demographics.show(5)

In [ ]:
demographics = demographics.drop(
    'State', 'District', 'Percent voting for Clinton')
demographics.show(5)

In [ ]:
demographics.scatter('College%', 'Median Income')

Slide: Discussion question

In [ ]:
correlation(demographics, 'College%', 'Median Income')

In [ ]:
regression_slope = slope(demographics, 'College%', 'Median Income')
regression_intercept = intercept(demographics, 'College%', 'Median Income')
regression_slope, regression_intercept

In [ ]:
def fitted_values(t, x, y):
    """Return an array of the regressions estimates at all the x values"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a*t.column(x) + b

In [ ]:
predicted = fitted_values(demographics, 'College%', 'Median Income')

In [ ]:
demographics = demographics.with_column(
    'Linear Prediction', predicted)
demographics.scatter('College%')

There are some errors!

In [ ]:
actual = demographics.column('Median Income')
errors = actual - predicted

In [ ]:
demographics.with_column('Error', errors)

Should we look at the average of errors?

In [ ]:
np.mean(errors)

In [ ]:
np.mean(errors ** 2) ** 0.5

We call this value the root mean square error.

In [ ]:
# function defined at the beginning of this section
# uses 4 points, draws a red line from point to it's predicted value 
# to visualize the error
demographics_errors(regression_slope, regression_intercept)

In [ ]:
# takes any slope, any intercept

demographics_errors(1500, 20000)

In [ ]:
demographics_errors(-1000, 75000)

Slide: Error in Estimation

### Root Mean Square Error ###

In [ ]:
def show_demographics_rmse(slope, intercept):
    demographics_errors(slope, intercept)
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    prediction = slope * x + intercept
    mse = np.mean((y - prediction) ** 2)
    print("Root mean squared error:", round(mse ** 0.5, 2))

In [ ]:
show_demographics_rmse(-1000, 75000)

In [ ]:
show_demographics_rmse(1500, 20000)

In [ ]:
show_demographics_rmse(regression_slope, regression_intercept)

Slide: Least Squares Line

### Numerical Optimization ###

In [ ]:
x = np.arange(1, 3, 0.1)
y = (x-2)**2 + 3
Table().with_columns('x', x, 'y', y).plot('x')

In [ ]:
def f(x):
    return ((x-2)**2) + 3

In [ ]:
minimize(f)

In [ ]:
x = np.arange(-1.5, 1.5, 0.05)
y2 = 2 * np.sin(x*np.pi) + x ** 3 + x ** 4 
Table().with_columns('x', x, 'y', y2).plot('x')

In [ ]:
def complicated_function(x):
    return 2 * np.sin(x*np.pi) + x ** 3 + x ** 4 

In [ ]:
minimize(complicated_function)

### Minimizing RMSE ###

In [ ]:
def demographics_rmse(any_slope, any_intercept):
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    estimate = any_slope*x + any_intercept
    return (np.mean((y - estimate) ** 2)) ** 0.5

In [ ]:
demographics_rmse(1500, 20000)

In [ ]:
demographics_rmse(-1000, 75000)

In [ ]:
minimize(demographics_rmse)

In [ ]:
make_array(regression_slope, regression_intercept)

### Nonlinear Regression ###

In [ ]:
shotput = Table.read_table('shotput.csv')
shotput

In [ ]:
shotput.scatter('Weight Lifted')

In [ ]:
def shotput_linear_rmse(any_slope, any_intercept):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = any_slope*x + any_intercept
    return np.mean((y - estimate) ** 2) ** 0.5

In [ ]:
best_line = minimize(shotput_linear_rmse)
best_line

In [ ]:
weights = shotput.column(0)

In [ ]:
linear_fit = best_line.item(0)*weights + best_line.item(1)

shotput.with_column(
    'Best Line', linear_fit
).scatter(0)

**Quadratic Function**

$$
f(x) ~=~ ax^2 + bx + c
$$
for constants $a$, $b$, and $c$.



In [ ]:
def shotput_quadratic_rmse(a, b, c):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = a*(x**2) + b*x + c
    return np.mean((y - estimate) ** 2) ** 0.5

In [ ]:
best_quad = minimize(shotput_quadratic_rmse)
best_quad

In [ ]:
# x = weight lifted = 100 kg
# Then predicted shot put distance:

(-0.00104)*(100**2) + 0.2827*100 - 1.5318

In [ ]:
quad_fit = best_quad.item(0)*(weights**2) + best_quad.item(1)*weights + best_quad.item(2)

In [ ]:
shotput.with_column('Best Quadratic Curve', quad_fit).scatter(0)